In [95]:
from html.parser import HTMLParser  
from urllib.request import urlopen  
from urllib import parse
import random
from bs4 import BeautifulSoup as bs
import re
from string import digits
import csv
import json

class LinkParser(HTMLParser):

    def handle_starttag(self, tag, attrs):
           if tag == 'a':
            for (key, value) in attrs:
                if key == 'href':
                    
                    newUrl = parse.urljoin(self.baseUrl, value)
 
                    self.links = self.links + [newUrl]

    def getLinks(self, url):
        self.links = []
  
        self.baseUrl = url

        response = urlopen(url)

     
        htmlBytes = response.read()

        htmlString = htmlBytes.decode("utf-8")
        self.feed(htmlString)
        return htmlString, self.links[4:16]
       
            
def randname():
   
    ''' returns a single random name from the us census'''
   
    fhand = urlopen('http://deron.meranda.us/data/census-derived-all-first.txt')
    table =  [line.strip() for line in fhand]
    tsplit = [t.split() for t in table]
    namebyte = [tsplit[i][0] for i in range(len(tsplit))]
    names = [i.decode('UTF-8') for i in namebyte]

    return random.choice(names)

def randword():
    
    ''' returns a random word from a list of 1524 of the most common nouns in the english language '''
    
    fhand = urlopen('http://www.talkenglish.com/vocabulary/top-1500-nouns.aspx').read()
    stringform = fhand.decode("utf-8")
    soup = bs(stringform, 'html.parser')
    tables = soup.find_all('tr')
    wordtags = tables[4:-1]
    words = [word.get_text() for word in wordtags]
    words = [w.strip() for w in words]
    words = [re.sub('\(.+\)', '', wo) for wo in words]
    # removes part of string in parentheses
    
    words = [w.translate({ord(k): None for k in digits}) for w in words]
    # removes numbers from each string
    words = [w.strip() for w in words]
    # removes whitespace around string
    return random.choice(words)
    
    

def searchaddress():
    
    ''' takes a random word and formats into URL, returns url1 for first search page 
        and url2 for second search page '''                     
    
    base = 'http://www.whosdatedwho.com/search?q='
    base2 = '&page=2'
    word = randword()
    url = base + word
    url2 = base + word + base2

    return url, url2, word

def picfinder(url):
    try:
        rawdata = urlopen(url).read()
        rawdata = rawdata.decode("utf-8")

        soup = bs(rawdata, 'html.parser')

        imgs = soup.find_all('img')
        srcs = ([img['src'] for img in imgs])
        picture = (srcs[1])

        if picture == 'http://img3.bdbphotos.com/images/130x130/5/g/5gmnjvqhgqy8jyh.jpg?skj2io4l':
            return 'http://vignette3.wikia.nocookie.net/spore/images/6/6c/Question-mark.png/revision/latest?cb=20110427230528'
        # profiles with no picture return selena gomez (current top celebrity)
        # so an image of a question mark is returned if selena gomez if found
        # could be updated to automatically find the top celebrity
        else:
            return picture
    except:
        return 'no pic'
    
    

def celebinfo(url):
    
    ''' returns a dictionary  containing information about the celebrity '''
    
    rawdata = urlopen(url).read()
    rawdata = rawdata.decode("utf-8")
    soup = bs(rawdata, 'html.parser')
    try:
        tables = soup.find_all('table')
        data = tables[1].find_all('td')

        keys = data[::2]
        keys = [k.get_text() for k in keys]
        keys = [k.strip() for k in keys]

        vals = data[1::2]
        vals = [v.get_text() for v in vals]
        vals = [v.strip() for v in vals]

        dictm = {k: v for k, v in zip(keys, vals)}
        
        imlink = picfinder(url)
        
        dictm['Image Source'] = imlink
        
        return dictm
    
    except:
        return 'empty'


def childage(url):
     
    ''' returns a list of the ages of each of the children '''
    
    rawdata = urlopen(url).read()
    rawdata = rawdata.decode("utf-8")

    soup = bs(rawdata, 'html.parser')

    tables = soup.find_all('table')

    data = tables[3].find_all('td')
    
    data = [d.get_text() for d in data]
    data = [d.strip() for d in data]
    
    agestr = [x for x in data if "years" in x ]
    
    ages = []
    
    for i in range(len(agestr)):
        ages.append([int(s) for s in agestr[i].split() if s.isdigit()])
        
    ages = [ages[i][0] for i in range(len(ages))]
    
    return ages


def string_found(string1, string2):
    if re.search(r"\b" + re.escape(string1) + r"\b", string2):
        return True
    return False

def bing_pic_finder(word):
    
    word = word.replace(' ','%20')
    rawdata = urlopen('http://www.bing.com/images/search?q=' + word + '&go=Search&qs=n&form=QBILPG&pq=' + word + '&sc=0-0&sp=-1&sk=').read()
    rawdata = rawdata.decode("utf-8")

    soup = bs(rawdata, 'html.parser')
    links = soup.find_all('a')

    hrefs = [link['href'] for link in links]
    pics = [href for href in hrefs if href.endswith(('.png', '.PNG', '.jpg', '.JPG'))] 

    return pics


def spider(): 
    print('Starting Spider')
    maxPages = 120
    numberVisited = 0
    interesting = False
    
    while numberVisited < maxPages and not interesting: 

        numberVisited = numberVisited + 1
        
        url1, url2, searchedword = searchaddress()
        
        urls = [url1, url2]
        
        for url in urls:
        
        
            parser = LinkParser()
            data, links = parser.getLinks(url)


            infos = [celebinfo(link) for link in links]
     
            for info in random.sample(infos, len(infos)):

                try:

                    firstname = info.get('First Name')
                    firstname = firstname + ' x'
    

                    if string_found(searchedworld.title(), firstname) == True or string_found('The', firstname) == True:
                        
                        interesting = True
                   
                        return info
                    else:
                        pass
                except:
                    pass


                try:

                    lastname = info.get('Last Name')
                    lastname = lastname + ' x'


                    if string_found(searchedword.title(), lastname) == True or string_found('The', lastname) == True:
                        
                        interesting = True
                        return info
                    else:
                        pass


                except:
                    pass

                try:

                    midname = info.get('Middle Name')
                    midname = midname + ' x'


                    if string_found(searchedword.title(), midname) == True or string_found('The', midname) == True:
                        
                        interesting = True
                        return info
                    else:
                        pass


                except:
                    pass


                
                
bing_pic_finder('thomas is gay')
# with open(r'gerardisgay.csv', 'a') as f:
#     for i in range(2000):
#         try:
#             x = spider()
#             f.write("%s\n" % x)
            
#         except:
#             print('poo')

# with open('gerard_is_very_gay.json', 'w') as outfile:
#     for i in range(2):
#         try:
#             data = spider()
#             json.dump(data, outfile, indent=4, sort_keys=True, separators=(',', ':'))
#         except:
#             print('poo')



['http://photos1.blogger.com/blogger/6296/2252/1600/untitled3.jpg',
 'http://i926.photobucket.com/albums/ad104/BigBlueCowboy/ThomasRoberts.jpg',
 'http://www.me-me-me.tv/images/2011/10/Thomas-Jane.jpg',
 'http://2.bp.blogspot.com/-_0r-vD8K3sM/T9mfvahapnI/AAAAAAAAINA/s7c4lFxiJGM/s1600/Tom+Cruise+gay.jpg',
 'http://4.bp.blogspot.com/-Cx3GTeCBqe8/TwX8NtezJGI/AAAAAAAAB-0/Mvx3cB275_w/s1600/gareth-thomas-1.jpg',
 'http://images.digitalspy.co.uk/08/16/w550_tom_welling.jpg',
 'http://www.pinknews.co.uk/images/2009/12/gareththomasrugby.jpg',
 'http://a.dilcdn.com/bl/wp-content/uploads/sites/8/2011/02/SGG-031755.jpg',
 'http://www.aceshowbiz.com/images/news/thomas-roberts-ties-the-knot-with-patrick-abner.jpg',
 'http://upload.wikimedia.org/wikipedia/commons/6/6e/Gareth_Thomas_(rugby_player).jpg',
 'http://en.wikipedia.org/wiki/File:Gareth_Thomas_(rugby_player).jpg',
 'http://www.worldoftomoffinland.com/tomsblog/wp-content/uploads/2012/09/Life-and-work-of-a-gay-hero-cover.jpg',
 'http://www.doubl